In [ ]:
# simsiam
!experiment_name="dcnet";cd ../ ;\
    name_date="${experiment_name}/$(date +%Y-%m%d-%H%M%S)";\
    dir="./results/${name_date}";mkdir -p ${dir};dir_log="${dir}/output.log";\
CUDA_VISIBLE_DEVICES=3,8 torchrun --nproc_per_node=2 --master_port=1234 train_multi_GPU.py \
    --wandb False --wandb_model dryrun --sync_bn False --amp True --aux False \
    --model_name deeplabv3_resnet101 --pre_trained deeplabv3_resnet101_coco.pth \
    --data_path cityscapes --num_classes 19 \
    --epochs 40 --batch_size 4 --batch_size_val 4 --memory_size 0 \
    --contrast 0 --loss_name simsiam --L1_loss 0.1 --L2_loss 0.1 --L3_loss 0.1\
    --name_date $name_date \
    2>&1 | tee $dir_log

In [8]:
# benchmark
!experiment_name="debug";cd ../ ;\
    name_date="${experiment_name}/$(date +%Y-%m%d-%H%M%S)";\
    dir="./results/${name_date}";mkdir -p ${dir};dir_log="${dir}/output.log";\
CUDA_VISIBLE_DEVICES=8 torchrun --nproc_per_node=1 --master_port=12345 train_multi_GPU.py \
    --wandb False --wandb_model run --sync_bn False --amp True --aux False \
    --batch_size 10 --batch_size_val 10 --memory_size 0 \
    --epochs 40 --data_path pascal-voc-2012 \
    --model_name deeplabv3_resnet101 --pre_trained pre-trained/deeplabv3_resnet101_coco.pth \
    --contrast -1 --loss_name double --L1_loss 0.1 --L2_loss 0.1 --L3_loss 0.1\
    --name_date $name_date \
    2>&1 | tee $dir_log

| distributed init (rank 0): env://
debug/2023-0314-002527
Namespace(data_path='pascal-voc-2012', device='cuda', num_classes=20, batch_size=10, batch_size_val=10, aux=False, start_epoch=0, epochs=40, sync_bn=False, workers=1, lr=0.0001, momentum=0.9, weight_decay=0.0001, print_freq=10, checkpoint_dir='./results/debug/2023-0314-002527', resume='', test_only=False, world_size=1, dist_url='env://', amp=True, seed=304, name_date='debug/2023-0314-002527', wandb=False, wandb_model='run', model_name='deeplabv3_resnet101', project_dim=128, loss_name='double', contrast=-1, pre_trained='pre-trained/deeplabv3_resnet101_coco.pth', L3_loss=0.1, L2_loss=0.1, L1_loss=0.1, GAcc=1, memory_size=0, proj_dim=128, network_stride=8, pixel_update_freq=10, ddp=False, rank=0, gpu=0, distributed=True, dist_backend='nccl')
Creating data loaders
Creating model
missing_keys:  []
unexpected_keys:  ['aux_classifier.0.weight', 'aux_classifier.1.weight', 'aux_classifier.1.bias', 'aux_classifier.1.running_mean', 'aux_c